## Connect to Rafal

In [ ]:
# load connection script
%run ../../etc/rafal_load.py

uiConnect

## Monitoring Module

### using monitoring_flat

In [ ]:
from ipywidgets import widgets
import plotly.graph_objects as go
import plotly.express as px
import re

In [ ]:
module= 'monitoring_flat'
catg= ['client_hostname', 'request_path', 'result_code', 'server_hostname',
       'server_id', 'username', 'module', 'pivot', 'calculateur', 'label']

metrics= ["query", "sumQueryDuration", "sumQueryMemoryUsage", 
          "sumQueryReadBytes", "sumQueryResultBytes", "sumQueryWrittenBytes"]
aggDict= {fld.replace('sum', '') if 'sum' in fld else 'Count': (fld, 'count' if fld == 'query' else 'sum') for fld in metrics}

my_uuid1= str(uuid4())
request_content = {
    "metrics": ["count","sumQueryDuration", "sumQueryMemoryUsage", 
               "sumQueryReadBytes", "sumQueryResultBytes", "sumQueryWrittenBytes"],
    "postaggregaggr": {},
    "with_facet": True,
    "filter_on_query": None,
    "batch": False,
    "batch_description": None,
    "currency": None,
    "pivot": "endpoint_database_queries",
    "by": ["access_time",
        "client_hostname",
        "request_id",
        "request_body",
        "request_path",
        "result_code",
        "server_hostname",
        "server_id",
        "username",
        "is_initial_query",
        "query",
        "query_start_time",
        "query_read_rows",
        "query_id",],
    "where": {
        "@and": [{"request_path": {"match": "^/module/"}}]
    },
    "versioning": [],
    "query_id":  my_uuid1
}

## plotly

## Figure Widgets

In [ ]:
def get_hour_freq(timeSerie):
    hl= [0.25, 0.5, 1, 2, 3, 6, 12, 24, 168]
    span= timeSerie.max()- timeSerie.min()
    period= max(int((span / pd.offsets.Hour()) / 60), hl[0])
    return [h for h in hl if h <= period][-1]

def monitoringRefresh(session):
    """get rafal_logging pivot response"""
    resp= session.request(endPoint= f'/module/{module}/query', http= 'POST', jsonText= request_content, 
                          output= 'json', verbose= False)

    # convert the data in Dataframe and filter monitoring request
    df0= pd.read_json(resp, orient='records',lines= True)
    df= df0[~df0.request_path.str.contains(r'monitoring')]
    # check unicity of query_id
    #assert (df['count'] == 1).all()

    df= df.assign(module= df.request_path.str.extract(r'^/module/(\w*)/', expand= False).fillna('-'),
                  pivot= df.request_body.str.extract(r'"pivot":"(\w*)"').fillna('-'),
                  calculateur= df.request_body.str.extract(r'"calculatorName":"(\w*)"').fillna('-'),
                  by= df.request_body.str.extract(r'"by":\[(?P<by>("\w*")*)\]', expand = False)['by'],
                  metrics= df.request_body.str.extract(r'"metrics":\[(?P<metrics>("\w*")*)\]', expand = False)['metrics'],
                  versioning= df.request_body.str.extract(r'"versioning":\[(?P<versioning>(.*))\]', expand = False)['versioning']
                 )
    df['label']= df.module + '/' + df['pivot'].str.replace('-', '') + df['calculateur'].str.replace('-', '') + (df['pivot']!='-').map({True:' piv'}).fillna((df['calculateur']!='-').map({True:' calc'}))
    # adjust sample frequency in Hour
    near_hour_freq= get_hour_freq(df.access_time)
    return df.groupby([*catg, pd.Grouper(key='access_time', freq= f'{near_hour_freq}H')]).agg(**aggDict).reset_index()

def getHostname(url):
    return re.match('http://(.*):', url).groups()[0]

def pxBuild(dfmon, metric):
    return px.bar(dfmon, x= wXaxis.value, y= metric, 
               hover_data=['Count', 'QueryDuration', 'QueryReadBytes', 'QueryMemoryUsage'], 
               color= wColor.value,
               labels={'QueryMemoryUsage':'Memory Usage in Bytes'}, height = 400)


def requestServer(change):
    if session.url != wServer.value.strip('/'):
        with wOutConnect:
            wOutConnect.clear_output()
            print(f'Please connect first to the server by clicking the [Connect] button !')
            return
    else:
        with wOutConnect:
            wOutConnect.clear_output()
            print(f'Requesting data from {session.url}.\nPlease wait ...', end=' ')
            dfmon= monitoringRefresh(session)
            print(dfmon.iloc[0]['server_hostname'])
            print(f'Done !')        

def validate():
    return True

def updFigure(change):
    if validate():
        pxFig= pxBuild(dfmon, wMetric.value)            
        new_data= {'x': [getattr(trace, 'x') for trace in pxFig.data],
                   'y': [getattr(trace, 'y') for trace in pxFig.data],
                  }        
        with gf.batch_update():
            gf.plotly_update(restyle_data= new_data, 
                             relayout_data= {'yaxis.title.text': wMetric.value,
                                             'xaxis.title.text': dfmon.iloc[0]['server_hostname']})

In [ ]:
dfmon= monitoringRefresh(session)

In [ ]:
wXaxis= widgets.Dropdown(
        description='X axis',
        options= ['access_time'] + catg,
        value= 'access_time'
    )
wColor= widgets.Dropdown(
        description='Color',
        options= catg,
        value= 'label'
    )
wMetric= widgets.Dropdown(
    description='metric',
    options= aggDict.keys(),
    value= 'Count'
)
bRetrieve = widgets.Button(description='Retrieve data', button_style='warning',)
bRetrieve.on_click(requestServer)
wConnect= widgets.HBox([widgets.Label('Rafal server'), wServer, button, bRetrieve])
wOutConnect= widgets.Output()
wDimensions= widgets.HBox([wXaxis, wColor])
out = widgets.Output(layout={'border': '1px solid black'})

wDimensions

In [ ]:
#bRetrieve._click_handlers.callbacks=[]
wMetric.observe(updFigure, names= "value")

#print(f'Requesting data from {wServer.value}.\nPlease wait ...')
#dfmon= monitoringRefresh(session)
#print(f'Computing monitoring data ...', end=' ')
pxFig= pxBuild(dfmon, wMetric.value)
#print(f'Done !')
gf= go.FigureWidget(data= pxFig.data, layout= pxFig.layout)
gf.layout.title.text = f'Monitoring on Rafal@{getHostname(wServer.value)}'
ui = widgets.VBox([wMetric, gf])

ui

In [ ]:
dfmon